In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ml-p2/sample_submission.csv
/kaggle/input/2021-ml-p2/x_test.csv
/kaggle/input/2021-ml-p2/y_train.csv
/kaggle/input/2021-ml-p2/x_train.csv


# [날씨 및 유동인구에 따른 교통량 추이](https://www.kaggle.com/c/2021-ml-p2)
> 기계학습 10주차 실습

In [2]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings('ignore')

In [3]:
file_dir = "/kaggle/input/2021-ml-p2/"

X_train = pd.DataFrame(pd.read_csv(file_dir + 'x_train.csv', engine='python', encoding='CP949'))
y_train = pd.DataFrame(pd.read_csv(file_dir + 'y_train.csv', engine='python', encoding='CP949'))
X_test = pd.DataFrame(pd.read_csv(file_dir + 'x_test.csv', engine='python', encoding='CP949'))
submission = pd.DataFrame(pd.read_csv(file_dir + 'sample_submission.csv', engine='python', encoding='CP949'))

In [4]:
submission.head()

,id,predict
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


## 데이터 확인

In [5]:
X_train.shape, X_test.shape

((21486, 14), (10583, 14))

In [6]:
y_train

,교통량
0,0.000
1,1532.989
2,710.107
3,374.200
4,535.882
...,...
21481,91.000
21482,0.000
21483,0.000
21484,283.625


In [7]:
np.sum(y_train < 0)

교통량    0
dtype: int64

In [8]:
np.sum(y_train == 0)

교통량    9022
dtype: int64

In [9]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21486 entries, 0 to 21485
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   시도명       21486 non-null  int64  
 1   읍면동명      21486 non-null  int64  
 2   거주인구      21486 non-null  float64
 3   근무인구      21486 non-null  float64
 4   방문인구      21486 non-null  float64
 5   총 유동인구    21486 non-null  float64
 6   평균 속도     21486 non-null  float64
 7   평균 소요 시간  21486 non-null  float64
 8   평균 기온     21486 non-null  float64
 9   일강수량      21486 non-null  float64
 10  평균 풍속     21486 non-null  float64
 11  월         21486 non-null  int64  
 12  일         21486 non-null  int64  
 13  년         21486 non-null  int64  
dtypes: float64(9), int64(5)
memory usage: 2.3 MB


In [10]:
X_train.head()

,시도명,읍면동명,거주인구,근무인구,방문인구,총 유동인구,평균 속도,평균 소요 시간,평균 기온,일강수량,평균 풍속,월,일,년
0,0,26,76018.965,5009.811,36887.341,117916.117,40.467,21.667,17.75,0.0,2.725,9,25,2019
1,1,24,1232416.968,76043.955,451558.268,1760019.191,38.126,31.632,13.30,14.5,4.500,3,4,2018
2,1,23,134260.946,8412.761,78177.981,220851.688,51.107,45.643,5.25,0.5,4.950,4,7,2018
3,1,25,289327.429,39102.424,287029.303,615459.156,33.067,48.000,26.30,0.5,1.500,8,14,2019
4,1,7,1116617.660,96560.651,524905.301,1738083.612,40.392,26.961,8.40,44.0,4.300,12,1,2019


## preprocessing

In [11]:
drop_cols = ['년', '일']
X_train_d = X_train.drop(drop_cols, axis=1)
X_test_d = X_test.drop(drop_cols, axis=1)

In [12]:
mm_scaler = MinMaxScaler()
mm_scaler.fit(X_train_d)
X_train_d_mm = mm_scaler.transform(X_train_d)
X_test_d_mm = mm_scaler.transform(X_test_d)

std_scaler = StandardScaler()
std_scaler.fit(X_train_d)
X_train_d_std = std_scaler.transform(X_train_d)
X_test_d_std = std_scaler.transform(X_test_d)

pca = PCA(n_components=7, whiten=True).fit(X_train)
X_train_pca = pca.transform(X_train)
X_test = pca.transform(X_test)


## 파라미터 조정없이 확인

### non scaled data

In [13]:
# scores = cross_validate(SVR(), X_train_d, y_train, cv=4,
#                        scoring = 'r2', return_train_score=True)

In [14]:
# scores

### scaled data

In [15]:
# scores = cross_validate(SVR(), X_train_d_mm, y_train, cv=4,
#                        scoring = 'r2', return_train_score=True)
# scores

In [16]:
# scores = cross_validate(SVR(), X_train_d_std, y_train, cv=4,
#                        scoring = 'r2', return_train_score=True)
# scores

### kernel

In [17]:
# scores = cross_validate(SVR(kernel='linear'), X_train_d_mm, y_train, cv=4,
#                        scoring = 'r2', return_train_score=True)
# scores

In [18]:
# # poly kernel이 가장 높았다.
# scores = cross_validate(SVR(kernel='poly'), X_train_d_mm, y_train, cv=4,
#                        scoring = 'r2', return_train_score=True)
# scores

In [19]:
# scores = cross_validate(SVR(kernel='sigmoid'), X_train_d_mm, y_train, cv=4,
#                        scoring = 'r2', return_train_score=True)
# scores

### pca data

In [20]:
# scores = cross_validate(SVR(kernel='poly'), X_train_pca , y_train, cv=4,
#                        scoring = 'r2', return_train_score=True)
# scores

In [21]:
# scores = cross_validate(SVR(kernel='rbf'), X_train_pca , y_train, cv=4,
#                        scoring = 'r2', return_train_score=True)
# scores

## parameter tuning

In [22]:
# scores = cross_validate(SVR(kernel='poly', degree=2), X_train_d_mm, y_train, cv=4,
#                        scoring = 'r2', return_train_score=True)
# scores

In [23]:
# scores = cross_validate(SVR(kernel='poly', degree=4), X_train_d_mm, y_train, cv=4,
#                        scoring = 'r2', return_train_score=True)
# scores

In [24]:
# scores = cross_validate(SVR(kernel='poly', degree=5), X_train_d_mm, y_train, cv=4,
#                        scoring = 'r2', return_train_score=True)
# scores

In [25]:
# scores = cross_validate(SVR(kernel='poly', degree=6), X_train_d_mm, y_train, cv=4,
#                        scoring = 'r2', return_train_score=True)
# scores

In [26]:
# scores = cross_validate(SVR(kernel='poly', degree=7), X_train_d_mm, y_train, cv=4,
#                        scoring = 'r2', return_train_score=True)
# scores

In [27]:
# scores = cross_validate(SVR(kernel='poly', degree=8), X_train_d_mm, y_train, cv=4,
#                        scoring = 'r2', return_train_score=True)
# scores

In [28]:
# scores = cross_validate(SVR(kernel='poly', degree=9), X_train_d_mm, y_train, cv=4,
#                        scoring = 'r2', return_train_score=True)
# scores

In [29]:
submission.head()

,id,predict
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [30]:
svr = SVR(kernel='poly', degree=10).fit(X_train_d_mm, y_train)

y_test_pred = svr.predict(X_test_d_mm)
y_test_pred[y_test_pred < 0] = 0
submission['predict'] = y_test_pred
submission.to_csv('submission.csv', index=False)